# Univariate Quadrature Rules

An $n$-point quadrature rule $\{w_k, t_k \}_{k=1}^n$ composed of weights $w_k$ and nodes $t_k$ satisfies
$$
\int f(t) w(t) \mathrm{d}t \approx \sum_{k=1}^{n} w_k f(t_k).
$$
The integration interval can be finite, infinite or half-finite.
Gauss-quadrature rules are specific quadrature rules that satisfy
$$
\int p(t) w(t) \mathrm{d}t = \sum_{k=1}^{n} w_k p(t_k),
$$
where $p$ is a polynomial of degree at most $2n-1$.
In other words, with a 3-point Gauss quadrature rule, polynomials of degree at most 5 can be integrated *exactly* simply by evaluating the polynomial at three points, and weigthing them appropriately.
Gauss quadrature rules are intricately related to orthogonal polynomials (relative to the weight function $w(t)$).

## Classical Orthogonal Polynomials
The Gauss quadrature rules for classical orthogonal polynomials have been studied (and implemented) extensively.
Our package relies on [FastGaussQuadrature.jl](https://github.com/ajt60gaibb/FastGaussQuadrature.jl) to compute them.
For example, the $n$-point quadrature rule for a Gaussian measure is obtained via

In [4]:
using Revise, PolyChaos
n = 10
q_gauss = Quad("gaussian",n)


Quadrature rule
name:		gaussian
N:		10
nodes		[-4.85946, -3.58182, -2.48433, -1.46599, -0.484936, 0.484936, 1.46599]...
weights		[4.31065e-6, 0.000758071, 0.0191116, 0.135484, 0.344642, 0.344642, 0.135484]...

Measure dλ(t)=w(t)dt
name:		gaussian
w(t):		PolyChaos.w_gaussian
dom:		(-Inf, Inf)
symmetric:	true


As for `OrthoPoly`, also `Quad` has a field `meas`, which specifies the underlying measure.
The quadrature rule is obtained by calling `nw()`

In [5]:
nw(q_gauss) # equivalently call q_gauss.nodes, q_gauss.weights

10×2 Array{Float64,2}:
 -4.85946   4.31065e-6 
 -3.58182   0.000758071
 -2.48433   0.0191116  
 -1.46599   0.135484   
 -0.484936  0.344642   
  0.484936  0.344642   
  1.46599   0.135484   
  2.48433   0.0191116  
  3.58182   0.000758071
  4.85946   4.31065e-6 

Let's now solve the following integral
$$
\int_{\mathbb{R}} (0.2 t^8 + 0.1 t^7 + 0.34 t^3 + 2 t^2) \, \frac{1}{\sqrt{2 \pi}} \, \exp \left( - \frac{t^2}{2} \right) \mathrm{d} t
$$

In [6]:
f(t) = 0.2*t^8+0.1t^7 + 0.34*t^3 + 2t^2
I1 = integrate(f,q_gauss) # equivalent to dot(weights,f.(weights))

# using QuadGK
# g(t) = f(t)*1/sqrt(2*pi)*exp(-0.5*t^2)
# I2 = quadgk(g,-Inf,Inf)[1]

22.999999999999808

If necessary, additional parameters of the underlying measure are passed in a dictionary.
This is the case, for example, for the beta distribution.

In [7]:
q_beta = Quad("beta01",n,Dict(:shape_a=>2.3,:shape_b=>3.2))
nw(q_beta)

10×2 Array{Float64,2}:
 0.0297857  0.00818123
 0.090266   0.0493143 
 0.177721   0.126308  
 0.28643    0.202984  
 0.409284   0.231663  
 0.538247   0.195276  
 0.664887   0.120539  
 0.780926   0.0514044 
 0.878792   0.0130437 
 0.95217    0.00128664

## Arbitrary Weights
For arbitrary weights $w(t)$ the quadrature rule can be computed numerically from the recursion coefficients of the underlying orthogonal polynomials.
Let's first define the measure (here the uniform measure on $(0,1)$), and compute the respective orthogonal polynomials.

In [ ]:
supp = (0,1)
function w(t)
    supp[1]<=t<=supp[2] ? (1.) : error("$t not in support")
end
my_meas = Measure("my_meas",w,supp,false,Dict())
my_op = OrthoPoly("my_op",n-1,my_meas;Nquad=200)

Using the Golub-Welsch algorithm by default, the quadrature rule is computed as follows

In [ ]:
my_q = Quad(n,my_op)

Let's compare the accuracy of the numerically computed quadrature rule to its well-studied Gauss-Legendre counterpart:

In [ ]:
q_uniform01 = Quad("uniform01",n)
nw(q_uniform01) - nw(my_q)

__The possibility to create quadrature rules for arbitrary weights should be reserved to cases different from *classical* ones.__

## Multivariate Quadrature Rules
No stand-alone implementation currently, the reason being that the standard use case of the package is to use orthogonal polynomials in conjunction with quadrature rules, see __here__.